Plot ATLAS Compact
==================

Check if a given point is within a tide model domain

OTIS format tidal solutions provided by Ohio State University and ESR  
- http://volkov.oce.orst.edu/tides/region.html  

#### Python Dependencies
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [netCDF4: Python interface to the netCDF C library](https://unidata.github.io/netcdf4-python/)  
 - [matplotlib: Python 2D plotting library](https://matplotlib.org/)  

#### Program Dependencies

- `convert_ll_xy.py`: convert lat/lon points to and from projected coordinates  
- `model.py`: retrieves tide model parameters for named tide models
- `read_tide_model.py`: extract tidal harmonic constants from OTIS tide models  

This notebook uses Jupyter widgets to set parameters for calculating the tidal maps.

#### Load modules

In [1]:
from __future__ import print_function

import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

import pyTMD.time
import pyTMD.model
import pyTMD.tools
import pyTMD.read_tide_model
# autoreload
%load_ext autoreload
%autoreload 2

In [3]:
# available model list
model_list = sorted(pyTMD.model.ATLAS_compact())
# display widgets for setting directory and model
TMDwidgets = pyTMD.tools.widgets()
TMDwidgets.model.options = model_list
TMDwidgets.model.value = 'TPXO8-atlas'
TMDwidgets.compress.value = False
widgets.VBox([
    TMDwidgets.directory,
    TMDwidgets.model,
    TMDwidgets.compress,
])

In [ ]:
# get model parameters
model = pyTMD.model(TMDwidgets.directory.value,
    compressed=TMDwidgets.compress.value
   ).elevation(TMDwidgets.model.value)
# read each constituent
constituents,nc = pyTMD.read_tide_model.read_constituents(model.model_file)

In [ ]:
# if reading a global solution with localized solutions
x0,y0,hz0,mz0,iob,dt,pmask,local = pyTMD.read_tide_model.read_atlas_grid(model.grid_file)
xi,yi,hz = pyTMD.read_tide_model.combine_atlas_model(x0,y0,hz0,pmask,local,VARIABLE='depth')
mz = pyTMD.read_tide_model.create_atlas_mask(x0,y0,mz0,local,VARIABLE='depth')
# resample global solution to 2 arc-minute solution
x30,y30,hz30 = pyTMD.read_tide_model.interpolate_atlas_model(x0,y0,hz0)

# plot the difference between ATLAS depth
fig,ax1 = plt.subplots(num=1, figsize=(8.25,5.25), dpi=200)
ax1.imshow(hz30 - hz, interpolation='nearest',
    extent=(xi.min(),xi.max(),yi.min(),yi.max()),
    vmin=-100, vmax=100, origin='lower', cmap='coolwarm')
# no ticks on the x and y axes
ax1.get_xaxis().set_ticks([])
ax1.get_yaxis().set_ticks([])
# stronger linewidth on frame
[i.set_linewidth(2.0) for i in ax1.spines.values()]
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
plt.show()

In [ ]:
# total amplitude difference for all constituents
diff = np.zeros_like(hz30)
# for each constituent
for i,c in enumerate(constituents):
    # if reading a global solution with localized solutions
    z0,zlocal = pyTMD.read_tide_model.read_atlas_elevation(model.model_file,i,c)
    xi,yi,z = pyTMD.read_tide_model.combine_atlas_model(x0,y0,z0,pmask,zlocal,VARIABLE='z')
    # resample global solution to 2 arc-minute solution
    x30,y30,z30 = pyTMD.read_tide_model.interpolate_atlas_model(x0,y0,z0)
    # add to total amplitude difference
    diff += (z30.real - z.real)**2 + (z30.imag - z.imag)**2

In [ ]:
# plot the difference between ATLAS tidal amplitudes
fig,ax1 = plt.subplots(num=1, figsize=(8.25,5.25), dpi=200)
ax1.imshow(np.sqrt(diff), interpolation='nearest',
    extent=(xi.min(),xi.max(),yi.min(),yi.max()),
    vmin=0, vmax=0.5, origin='lower', cmap='Reds')
# no ticks on the x and y axes
ax1.get_xaxis().set_ticks([])
ax1.get_yaxis().set_ticks([])
# stronger linewidth on frame
[i.set_linewidth(2.0) for i in ax1.spines.values()]
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
plt.show()